<a href="https://colab.research.google.com/github/Mehdi007bond/Predictive_maintenance_Project/blob/main/Predictive_maintenance_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I started by generating the dataset so i can work


In [7]:
import pandas as pd
import numpy as np
import datetime

# --- 1. Configuration Générale ---
N_LINES = 4  # Nombre de lignes de production à simuler (ex: Line_1, Line_2, ...)
DAYS_PER_MACHINE = 60
SAMPLES_PER_HOUR = 4
N_SAMPLES_PER_DAY = 24 * SAMPLES_PER_HOUR

# Paramètres de simulation de panne
FAILURE_WINDOW_HOURS = 24
FAILURE_HORIZON_HOURS = 12

# --- 2. Profils des Machines ---
def get_machine_parameters(machine_type):
    """Définit les profils de capteurs et de défaillance pour chaque type de machine."""

    if machine_type == 'Fraiseuse':
        # Haute charge : vibrations, couple et courant élevés
        return {
            'BASE_TEMP': 45.0, 'TEMP_NOISE': 2.0, 'TEMP_DEGRADE_RATE': 0.1,
            'BASE_VIB': 1.2, 'VIB_NOISE': 0.3, 'VIB_DEGRADE_RATE': 0.1,
            'BASE_CUR': 10.0, 'CUR_NOISE': 0.5, 'CUR_DEGRADE_RATE': 0.08,
            'BASE_TORQUE': 80.0, 'TORQUE_NOISE': 3.0, 'TORQUE_DEGRADE_RATE': 0.15
        }
    elif machine_type == 'Convoyeur':
        # Faible charge : stable, défaillance par friction/bourrage
        return {
            'BASE_TEMP': 30.0, 'TEMP_NOISE': 1.0, 'TEMP_DEGRADE_RATE': 0.05,
            'BASE_VIB': 0.4, 'VIB_NOISE': 0.1, 'VIB_DEGRADE_RATE': 0.03,
            'BASE_CUR': 3.0, 'CUR_NOISE': 0.1, 'CUR_DEGRADE_RATE': 0.15,
            'BASE_TORQUE': 20.0, 'TORQUE_NOISE': 1.0, 'TORQUE_DEGRADE_RATE': 0.05
        }
    elif machine_type == 'Machine_de_finition':
        # Très faible charge : précision, défaillance par déséquilibre
        return {
            'BASE_TEMP': 35.0, 'TEMP_NOISE': 0.5, 'TEMP_DEGRADE_RATE': 0.03,
            'BASE_VIB': 0.2, 'VIB_NOISE': 0.05, 'VIB_DEGRADE_RATE': 0.08,
            'BASE_CUR': 2.0, 'CUR_NOISE': 0.1, 'CUR_DEGRADE_RATE': -0.05,
            'BASE_TORQUE': 10.0, 'TORQUE_NOISE': 0.5, 'TORQUE_DEGRADE_RATE': -0.02
        }
    else:
        raise ValueError("Type de machine inconnu")

# --- 3. Fonction de Génération (Mise à jour) ---
def generate_machine_data(machine_id, machine_type, production_line, start_date):

    print(f"Génération des données pour Machine ID: {machine_id} (Type: {machine_type}, Ligne: {production_line})...")

    params = get_machine_parameters(machine_type)
    total_samples = DAYS_PER_MACHINE * N_SAMPLES_PER_DAY

    # Ajout d'un petit décalage en secondes pour garantir des timestamps uniques
    time_offset_seconds = np.random.randint(0, 30)
    base_timestamps = [start_date + datetime.timedelta(hours=i/SAMPLES_PER_HOUR) for i in range(total_samples)]
    timestamps = [ts + datetime.timedelta(seconds=time_offset_seconds) for ts in base_timestamps]

    # Initialisation des capteurs
    temp = np.random.normal(loc=params['BASE_TEMP'], scale=params['TEMP_NOISE'], size=total_samples)
    vib = np.random.normal(loc=params['BASE_VIB'], scale=params['VIB_NOISE'], size=total_samples)
    curr = np.random.normal(loc=params['BASE_CUR'], scale=params['CUR_NOISE'], size=total_samples)
    torque = np.random.normal(loc=params['BASE_TORQUE'], scale=params['TORQUE_NOISE'], size=total_samples)

    # Simulation de Panne
    failure_sample = total_samples - np.random.randint(N_SAMPLES_PER_DAY * 3, N_SAMPLES_PER_DAY * 15)
    degradation_start_sample = failure_sample - (FAILURE_WINDOW_HOURS * SAMPLES_PER_HOUR)

    for i in range(degradation_start_sample, failure_sample):
        progress = (i - degradation_start_sample) / (failure_sample - degradation_start_sample)
        temp[i] += params['TEMP_DEGRADE_RATE'] * progress * 20
        vib[i] += params['VIB_DEGRADE_RATE'] * progress * 15
        curr[i] += params['CUR_DEGRADE_RATE'] * progress * 10
        torque[i] += params['TORQUE_DEGRADE_RATE'] * progress * 10

    # Création des Cibles
    failure = np.zeros(total_samples, dtype=int)
    failure[failure_sample] = 1
    failure_imminent = np.zeros(total_samples, dtype=int)
    prediction_window_start = failure_sample - (FAILURE_HORIZON_HOURS * SAMPLES_PER_HOUR)
    failure_imminent[prediction_window_start:failure_sample] = 1
    rul = np.full(total_samples, fill_value=9999)
    for i in range(failure_sample):
        rul[i] = (failure_sample - i) / SAMPLES_PER_HOUR
    rul[failure_sample:] = 0

    # Assemblage du DataFrame
    df = pd.DataFrame({
        'timestamp': timestamps,
        'production_line': production_line, # NOUVELLE COLONNE
        'machine_id': machine_id,
        'machine_type': machine_type,
        'temperature': temp.round(2),
        'vibration': vib.round(4),
        'current': curr.round(3),
        'torque': torque.round(2),
        'failure': failure,
        'failure_imminent': failure_imminent,
        'RUL_hours': rul
    })

    return df

# --- 4. Boucle Principale de Génération ---
print("Démarrage de la génération du dataset de la ligne de production...")
all_data_frames = []
start_date = datetime.datetime(2024, 1, 1)
machine_id_counter = 1
machine_types = ['Fraiseuse', 'Convoyeur', 'Machine_de_finition']

for i in range(N_LINES):
    line_name = f"Line_{i+1}"
    print(f"\n--- Génération des données pour {line_name} ---")
    for machine_type in machine_types:
        machine_df = generate_machine_data(
            machine_id=machine_id_counter,
            machine_type=machine_type,
            production_line=line_name,
            start_date=start_date
        )
        all_data_frames.append(machine_df)
        machine_id_counter += 1

# --- 5. Finalisation : Mélange et Sauvegarde ---
print("\nFinalisation du dataset...")
# Concaténer tous les dataframes
full_dataset = pd.concat(all_data_frames)

# !! ÉTAPE CRUCIALE : TRIER PAR TIMESTAMP !!
print("Tri du dataset par timestamp pour simuler la collecte 'temps réel'...")
full_dataset.sort_values(by='timestamp', inplace=True)

# Réinitialiser l'index
full_dataset.reset_index(drop=True, inplace=True)

print("\nDataset de la ligne de production généré et mélangé !")
print(f"Nombre total d'échantillons: {len(full_dataset)}")

# Sauvegarde
output_filename = "production_line_maintenance_data_realtime.csv"
full_dataset.to_csv(output_filename, index=False)
print(f"Dataset sauvegardé sous '{output_filename}'")

# Affichage d'un échantillon pour montrer le mélange
print("\n--- Aperçu du Dataset (mélangé 'temps réel') ---")
print(full_dataset.head(10)) # On voit maintenant différentes machine_id

print("\n--- Aperçu de la fin du Dataset ---")
print(full_dataset.tail(10))

print("\n--- Répartition des Lignes de Production ---")
print(full_dataset['production_line'].value_counts())

Démarrage de la génération du dataset de la ligne de production...

--- Génération des données pour Line_1 ---
Génération des données pour Machine ID: 1 (Type: Fraiseuse, Ligne: Line_1)...
Génération des données pour Machine ID: 2 (Type: Convoyeur, Ligne: Line_1)...
Génération des données pour Machine ID: 3 (Type: Machine_de_finition, Ligne: Line_1)...

--- Génération des données pour Line_2 ---
Génération des données pour Machine ID: 4 (Type: Fraiseuse, Ligne: Line_2)...
Génération des données pour Machine ID: 5 (Type: Convoyeur, Ligne: Line_2)...
Génération des données pour Machine ID: 6 (Type: Machine_de_finition, Ligne: Line_2)...

--- Génération des données pour Line_3 ---
Génération des données pour Machine ID: 7 (Type: Fraiseuse, Ligne: Line_3)...
Génération des données pour Machine ID: 8 (Type: Convoyeur, Ligne: Line_3)...
Génération des données pour Machine ID: 9 (Type: Machine_de_finition, Ligne: Line_3)...

--- Génération des données pour Line_4 ---
Génération des données po

In [8]:
full_dataset.head(50)

,timestamp,production_line,machine_id,machine_type,temperature,vibration,current,torque,failure,failure_imminent,RUL_hours
0,2024-01-01 00:00:03,Line_1,3,Machine_de_finition,34.94,0.1647,1.858,10.31,0,0,1215
1,2024-01-01 00:00:03,Line_2,5,Convoyeur,30.71,0.4184,3.022,21.46,0,0,1162
2,2024-01-01 00:00:09,Line_3,8,Convoyeur,29.50,0.3834,3.043,19.23,0,0,1225
3,2024-01-01 00:00:09,Line_3,9,Machine_de_finition,36.23,0.2422,1.953,10.61,0,0,1252
4,2024-01-01 00:00:11,Line_3,7,Fraiseuse,45.70,1.2288,9.878,78.54,0,0,1224
5,2024-01-01 00:00:13,Line_4,11,Convoyeur,31.16,0.6811,3.144,18.69,0,0,1327
6,2024-01-01 00:00:14,Line_2,6,Machine_de_finition,35.10,0.1596,1.921,10.09,0,0,1194
7,2024-01-01 00:00:19,Line_2,4,Fraiseuse,43.05,1.2201,10.353,79.09,0,0,1274
8,2024-01-01 00:00:21,Line_4,10,Fraiseuse,43.56,0.8018,9.326,76.11,0,0,1238
9,2024-01-01 00:00:27,Line_4,12,Machine_de_finition,34.67,0.2362,1.958,10.01,0,0,1271
